In [6]:
# Exercise 02  —  Preprocessing
# ----------------------------------------------------------
import pandas as pd
from pathlib import Path

DATA_DIR = Path("..") / "data"
SRC_FILE  = DATA_DIR / "auto.csv"

auto = pd.read_csv(SRC_FILE, index_col="ID")

initial_cnt = auto.count()

auto = auto.drop_duplicates(
    subset=["CarNumber", "Make_n_model", "Fines"],
    keep="last"
)
after_dup_cnt = auto.count()

na_before = auto.isna().sum()

auto = auto.dropna(axis=1, thresh=len(auto) - 500)


auto["Refund"] = auto["Refund"].ffill()

auto["Fines"] = auto["Fines"].fillna(auto["Fines"].mean())

na_after = auto.isna().sum()

def split_make_model(s):
    parts = s.split(maxsplit=1)
    return pd.Series({
        "Make":  parts[0],
        "Model": parts[1] if len(parts) > 1 else None
    })

auto[["Make", "Model"]] = auto["Make_n_model"].apply(split_make_model)
auto = auto.drop(columns="Make_n_model")

DST_JSON = DATA_DIR / "auto.json"
auto.to_json(
    DST_JSON,
    orient="records",
    indent=2
)


tests:

In [7]:
df2 = pd.read_json('../data/auto.json', orient='records')

In [8]:
df2.count()

CarNumber    725
Refund       725
Fines        725
Make         725
Model        716
dtype: int64

In [9]:
df2 ['Fines'].mean()

8594.586466165412

In [10]:
df2 ['Refund'].mean()

1.5172413793103448